In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
from matplotlib.image import imread
import math
import cv2
import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [2]:
from tensorflow.keras.models import load_model

In [3]:
autoencoder = load_model("Autoencoder v3.h5")

In [4]:
autoencoder.summary()

Model: "sequential_11"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
sequential_7 (Sequential)    (None, 100, 100, 3)       931       
_________________________________________________________________
sequential_10 (Sequential)   (None, 400, 400, 1)       1983      
Total params: 2,914
Trainable params: 2,914
Non-trainable params: 0
_________________________________________________________________


In [5]:
autoencoder.layers

In [6]:
encoder = autoencoder.layers[0]

In [7]:
encoder.summary()

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_17 (Conv2D)           (None, 400, 400, 32)      544       
_________________________________________________________________
max_pooling2d_9 (MaxPooling2 (None, 200, 200, 32)      0         
_________________________________________________________________
conv2d_18 (Conv2D)           (None, 200, 200, 3)       387       
_________________________________________________________________
max_pooling2d_10 (MaxPooling (None, 100, 100, 3)       0         
Total params: 931
Trainable params: 931
Non-trainable params: 0
_________________________________________________________________


In [8]:
train_df = pd.read_csv("training_labels_fixed_fn.csv")

In [9]:
train_df

,name,Bicycle,Boat,Bottle,Bus,Car,Cat,Chair,Cup,Dog,Motorbike,People,Table
0,2015_00001.png,1,0,0,0,0,0,0,0,0,0,0,0
1,2015_00002.png,1,0,0,0,1,0,0,0,0,0,0,0
2,2015_00003.png,1,0,0,1,0,0,1,0,0,0,0,0
3,2015_00004.jpg,1,0,0,0,0,0,0,0,0,0,0,0
4,2015_00005.jpg,1,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5883,2015_07251.jpg,0,0,0,0,0,0,1,0,0,0,1,1
5884,2015_07252.jpg,0,0,0,0,0,0,0,0,0,0,0,1
5885,2015_07253.jpg,0,0,0,0,0,0,1,0,0,0,0,1
5886,2015_07254.jpg,0,0,0,0,0,0,1,0,0,0,0,1


In [10]:
test_df = pd.read_csv("testing_labels_fixed_fn.csv")

In [11]:
test_df

,name,Bicycle,Boat,Bottle,Bus,Car,Cat,Chair,Cup,Dog,Motorbike,People,Table
0,2015_00523.jpg,1,0,0,0,0,0,0,0,0,0,0,0
1,2015_00524.jpg,1,0,0,0,1,0,0,0,0,0,0,0
2,2015_00525.jpg,1,0,0,0,0,0,0,0,0,0,0,0
3,2015_00526.jpg,1,0,0,0,0,0,0,0,0,0,0,0
4,2015_00527.jpg,1,0,0,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1465,2015_07352.png,0,0,0,0,0,0,1,0,0,0,1,1
1466,2015_07353.png,0,0,0,0,0,0,0,0,0,0,1,1
1467,2015_07354.jpg,0,0,0,0,0,0,0,0,0,0,1,1
1468,2015_07355.jpg,0,0,0,0,0,0,1,0,0,0,0,1


In [12]:
columns = ['Bicycle', 'Boat', 'Bottle', 'Bus', 'Car', 
           'Cat', 'Chair','Cup', 'Dog', 'Motorbike', 'People', 'Table']

In [13]:
# Image Data Generator

In [14]:
def preprocessing_fun(img):
    img1 = img.astype('uint8')
    eq_img = cv2.equalizeHist(img1)
    return img

In [15]:
img_datagen = ImageDataGenerator(rescale=1./255,
                                 preprocessing_function = preprocessing_fun,
                                 horizontal_flip=True,
                                 brightness_range=[0.8,1.5],
                                 rotation_range=10)

In [16]:
test_img_datagen = ImageDataGenerator(rescale=1./255,
                                 preprocessing_function = preprocessing_fun)

In [17]:
data_dir = 'C:\\Users\\91882\\Desktop\\PA review 1\\ExDark\\Dataset For Model\\'

In [18]:
train_path = data_dir+'train\\'
test_path = data_dir+'test\\'

Reference for ImageDataGenerator: https://vijayabhaskar96.medium.com/multi-label-image-classification-tutorial-with-keras-imagedatagenerator-cd541f8eaf24

In [19]:
training_gen = img_datagen.flow_from_dataframe(dataframe=train_df,
                                               directory=train_path,
                                               x_col="name",
                                               y_col=columns,
                                               target_size=(400,400),
                                               color_mode='grayscale',
                                               batch_size=16,
                                               class_mode='raw',
                                               shuffle=False,
                                              validate_filenames=True)

Found 5888 validated image filenames.


In [20]:
testing_gen = test_img_datagen.flow_from_dataframe(dataframe=test_df,
                                              directory=test_path,
                                              x_col="name",
                                              y_col=columns,
                                              target_size=(400,400),
                                              color_mode='grayscale',
                                              batch_size=16,
                                              class_mode='raw',
                                              shuffle=False,
                                              validate_filenames=True
                                             )

Found 1470 validated image filenames.


In [21]:
# CNN Model

In [22]:
from tensorflow.keras.models import Sequential

In [23]:
from tensorflow.keras.layers import Conv2D, Dropout, MaxPooling2D,BatchNormalization,Activation,Flatten,Dense

In [24]:
encoder.output.shape[1:]

TensorShape([100, 100, 3])

In [25]:
cnn = Sequential()

cnn.add(Conv2D(32, (3, 3), padding='same',input_shape=(100,100,3)))
cnn.add(Activation('relu'))
cnn.add(MaxPooling2D(pool_size=(2, 2)))

cnn.add(Conv2D(32, (3, 3)))
cnn.add(Activation('relu'))
cnn.add(MaxPooling2D(pool_size=(2, 2)))
cnn.add(BatchNormalization())

cnn.add(Conv2D(64, (3, 3), padding='same'))
cnn.add(Activation('relu'))
cnn.add(MaxPooling2D(pool_size=(2, 2)))

cnn.add(Conv2D(64, (3, 3)))
cnn.add(Activation('relu'))
cnn.add(MaxPooling2D(pool_size=(2, 2)))
cnn.add(BatchNormalization())

cnn.add(Flatten())

cnn.add(Dense(512))
cnn.add(Activation('relu'))
cnn.add(Dropout(0.5))

cnn.add(Dense(12, activation='sigmoid'))

In [26]:
cnn.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 100, 100, 32)      896       
_________________________________________________________________
activation (Activation)      (None, 100, 100, 32)      0         
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 50, 50, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 48, 48, 32)        9248      
_________________________________________________________________
activation_1 (Activation)    (None, 48, 48, 32)        0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 24, 24, 32)        0         
_________________________________________________________________
batch_normalization (BatchNo (None, 24, 24, 32)        1

In [27]:
model = Sequential([encoder,cnn])

In [28]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
sequential_7 (Sequential)    (None, 100, 100, 3)       931       
_________________________________________________________________
sequential (Sequential)      (None, 12)                891820    
Total params: 892,751
Trainable params: 892,559
Non-trainable params: 192
_________________________________________________________________


In [29]:
encoder.trainable = False

In [30]:
model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])

In [31]:
from tensorflow.keras.callbacks import EarlyStopping
early_stop = EarlyStopping(monitor='val_loss',patience=3)

In [32]:
model.fit_generator(generator=training_gen,epochs=25,validation_data=testing_gen)

C:\Users\91882\anaconda3\envs\gputest\lib\site-packages\keras\engine\training.py:1972: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/25
368/368 [==============================] - 142s 367ms/step - loss: 0.4500 - accuracy: 0.0988 - val_loss: 0.3912 - val_accuracy: 0.0741
Epoch 2/25
368/368 [==============================] - 137s 371ms/step - loss: 0.3849 - accuracy: 0.1148 - val_loss: 0.4734 - val_accuracy: 0.0796
Epoch 3/25
368/368 [==============================] - 134s 364ms/step - loss: 0.3678 - accuracy: 0.1425 - val_loss: 0.3832 - val_accuracy: 0.0980
Epoch 4/25
368/368 [==============================] - 133s 360ms/step - loss: 0.3583 - accuracy: 0.1554 - val_loss: 0.4298 - val_accuracy: 0.0755
Epoch 5/25
368/368 [==============================] - 133s 361ms/step - loss: 0.3470 - accuracy: 0.1714 - val_loss: 0.3741 - val_accuracy: 0.0694
Epoch 6/25
368/368 [==============================] - 147s 400ms/step - loss: 0.3375 - accuracy: 0.1917 - val_loss: 0.5074 - val_accuracy: 0.0952
Epoch 7/25
368/368 [==============================] - 134s 364ms/step - loss: 0.3302 - accuracy: 0.2157 - val_loss: 0.4030 -

In [33]:
one_img = testing_gen.next()[1]

In [34]:
one_img.shape

(16, 12)